In [7]:
import cv2
print(cv2.__version__)

ImportError: libX11.so.6: cannot open shared object file: No such file or directory

In [6]:
import cv2
from ultralytics import YOLO
import threading
from queue import Queue
import time
import import_ipynb
from rnn import RNN
from tracker import TRACKER

# === CONFIGURATION ===
MODEL_PATH = "runs/detect/train/weights/best.pt" 
VIDEO_PATH = "raw_video/21.MOV"
CONFIDENCE_THRESHOLD = 0.3
SAVE_OUTPUT = False  
OUTPUT_PATH = "output_video.mp4"
SCALE_PERCENT = 30 
SHOW_FPS = True

ORANGE = (0,150,220)
BLUE = (175, 0, 0)
GREEN = (0, 150, 0)
RED = (0, 0, 175)
COLORS = [GREEN, RED, BLUE, ORANGE]

# === INITIALIZE ===
rnn = RNN()
tracker = TRACKER(30)

# === LOAD MODEL ===
model = YOLO(MODEL_PATH)
model.fuse() 
if model.device.type == 'cuda':
    model.half() 

# === QUEUE TO HOLD FRAMES FOR DISPLAY ===
frame_queue = Queue(maxsize=1) 

def display_thread():
    while True:
        try:
            frame = frame_queue.get()
            if frame is None:
                print("Frame is None. Exiting thread.")
                break
            print("Displaying frame...")
            cv2.imshow("YOLOv8 Detection", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("User requested exit.")
                break
        except Exception as e:
            print(f"Error in display thread: {e}")
    cv2.destroyAllWindows()


# === START VIDEO CAPTURE ===
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print(f"Could not open video: {VIDEO_PATH}")
    exit()

# === GET INITIAL FRAME SIZE AND FPS ===
ret, frame = cap.read()
if not ret:
    print("Failed to read video.")
    exit()

# === RESIZE SETUP ===
width = int(frame.shape[1] * SCALE_PERCENT / 100)
height = int(frame.shape[0] * SCALE_PERCENT / 100)

# === START DISPLAY THREAD ===
thread = threading.Thread(target=display_thread, daemon=True)
thread.start()

# === INFERENCE LOOP ===
import time
fps_counter = []
print("Starting inference...")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (width, height))

    start_time = time.time()

    # Run inference
    results = model(frame, conf=CONFIDENCE_THRESHOLD, iou=0.3) # iou intersection over union (supress bounding box overlay)
    annotated = results[0].plot()
    for result in results:
        boxes = result.boxes  # bounding boxes object

        if boxes is not None:
            centers = boxes.xywh[:, :2] 
            for center in centers:
                coord = tuple(map(int, center)) # coord = (x, y)
                tracker.track(coord)
                '''
                KLASSA HÄR ??????????????
                '''
                #cls = rnn.update(coord)  
    
    for id in tracker.trackedObjects:
        for coord in tracker.trackedObjects[id].trajectory:
            cv2.circle(annotated, coord, 1, COLORS[id%4])

        latestCoord = tracker.trackedObjects[id].trajectory[0]
        cv2.putText(annotated, f"{id}", latestCoord,
                    cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS[id%4], 1)

    if SHOW_FPS:
        fps = 1.0 / (time.time() - start_time)
        fps_counter.append(fps)
        cv2.putText(annotated, f"{fps:.1f} FPS", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,200), 2)


    if not frame_queue.full():
        frame_queue.put(annotated)


# === CLEANUP ===
frame_queue.put(None)
cap.release()
thread.join()

if SHOW_FPS and fps_counter:
    avg_fps = sum(fps_counter) / len(fps_counter)
    print(f"Average FPS: {avg_fps:.2f}")

ImportError: libX11.so.6: cannot open shared object file: No such file or directory